How to use the no reference image quality assessment to get efficient storege in remote sensing.
====================

This work studies the relationships between the intrinsec features of images and the degradation caused by compaction.  We use 3 modules of software to get this relationships, and 2 modlues to use in application. 

Below is an example of this relationship, 




In [0]:
from IPython.display import Image
from IPython.core.display import HTML 

In [30]:
Image(url= "https://raw.githubusercontent.com/rgiostri/Scipy_2019/master/graphics/index_1.png")

In the graphics, the different colors are different bit rates in JPEG2000 compactation.

This example use [Kakadu](http://kakadusoftware.com/) in compression module, the SSIM in quality module and Factal Dimension Gray Scale in No-Reference module.

In the right chart, the $SSIM_R$ is the value of SSIM with bit-rate 2, this value is limit of visually lossless.

Exploring this relationship it is possible to construct strategies to optimize memory,  below we make a sigmoid function training with [suburb images](https://senseflycom.s3.amazonaws.com/datasets/soda-hq/rgb-images.zip).


In [31]:
Image(url= "https://raw.githubusercontent.com/rgiostri/Scipy_2019/master/graphics/index_2.png")

The Disk Space is proportional to Bit-Rate.

Using this simple strategy is possible reduce the save 70% of disk space in relacion the bit-rate 2, ensuring the desired accuracy testing with 3 different datasets.


Sample of code:
================

Here are 3 meter options for use in the No-Reference module.


In [0]:
import numpy as np

from skimage import data
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte

import time

def timer(start,end):
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))  
    
    


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
data_folder='/content/drive/My Drive/Colab Notebooks/sample_images'

In [0]:
image_sample=img_as_ubyte(rgb2gray(data.imread(fname=data_folder+"/EP-00-00012_0119_0001.bmp")))

Fractal dimension gray (FDG) - With box counting
=========================

Code inspired by page:  https://francescoturci.net/2016/03/31/box-counting-in-numpy/

More about FDG:  https://link.springer.com/chapter/10.1007/978-981-10-7871-2_22


Simple Loop vs Numpy  functions vs Parallell Split Image with joblib
----------------------

Simple Loop vesions

In [0]:
#
# simpler and slower
#

def fractal_dimension_gray(image,M=256):
    
    # Only for 2d image
    assert(len(image.shape) == 2)
    dim_h,dim_v=image.shape
    
    #Indices in 3D structure
    ind=[]
    for i in range(dim_h):
        for j in range(dim_v):
            if image[i,j]>0:
                ind.append((i,j,Z_2[i,j]))
    ind=np.array(ind)

    # Minimal dimension of image
    p = min((dim_h,dim_v,M))

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 0, -2)

    # Actual box counting with decreasing size
    Ns = []
    for size in sizes:
        H, edges=np.histogramdd(ind, bins=(np.arange(0,dim_h+size,size),np.arange(0,dim_v+size,size),np.arange(0,M+1,size)))
        Ns.append(np.sum(H>0))

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(Ns), 1)
    return -coeffs[0]


In [0]:
# time spent for once
a=time.time()
print fractal_dimension_gray(image_sample)
b=time.time()
timer(a,b)

2.4055603640540757
00:01:02.39


Numpy funcitions version

In [0]:
#
# Replace the fist loop to make a 3D strutire
#
def image_3d_no_zero(image):
  #flags
  assert(len(image.shape) == 2)
  #only no zero index
  mask_no_0=image>0
  n_dif_zero=np.sum(mask_no_0)
  # no zero image
  assert(n_dif_zero > 0)
  image_single_index=image[mask_no_0].reshape(n_dif_zero)
  index=np.argwhere(mask_no_0).T
  return np.vstack((index,image_single_index)).T

In [0]:
#
# Separating the For of scales - Is possible parallel this using joblib or numba or other packages
#

def for_scales(im3d,scales,bins_lim):
    Ns=[]
    for size in scales:
        H, edges=np.histogramdd(im3d, bins=(np.arange(0,bins_lim[0]+size,size),np.arange(0,bins_lim[1]+size,size),np.arange(0,bins_lim[2]+size,size)))
        Ns.append(np.sum(H>0))
    return Ns

In [0]:
# Split in blocks of code


def fractal_dimension_gray_np(Z_2,M=256):
    
    dim_h,dim_v=Z_2.shape
    
    #Indices in 3D structure
    ind=image_3d_no_zero(Z_2)

    # Minimal dimension of image
    p = min((dim_h,dim_v,M))

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 0, -2)
    
    # Actual box counting with decreasing size
    
    Ns=for_scales(ind,sizes,(dim_h,dim_v,M))
    
    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(Ns), 1)
    return -coeffs[0]

In [0]:
#  time spent for once
a=time.time()
print fractal_dimension_gray_np(image_sample)
b=time.time()
timer(a,b)

2.4055603640540757
00:00:09.60


Paralleling the scales  using joblib

In [0]:
from joblib import Parallel, delayed

In [0]:
#
# Without loop
#
def core_for_scales(im3d,size,bins_lim):
    H, edges=np.histogramdd(im3d, bins=(np.arange(0,bins_lim[0]+size,size),np.arange(0,bins_lim[1]+size,size),np.arange(0,bins_lim[2]+size,size)))
    return np.sum(H>0)

In [0]:
# Split in blocks of code
# Note about Number of cores : More cores => More memory spent


def fractal_dimension_gray_np_parallel_scale(Z_2,M=256,cores=2):
    
    dim_h,dim_v=Z_2.shape
    
    #Indices in 3D structure
    ind=image_3d_no_zero(Z_2)

    # Minimal dimension of image
    p = min((dim_h,dim_v,M))

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 0, -2)    
    
        
    # Actual box counting with decreasing size
    
    Ns=Parallel(n_jobs=cores)(delayed(core_for_scales)(ind,node,(dim_h,dim_v,M)) for node in sizes)
    
    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(Ns), 1)
    return -coeffs[0]

In [0]:
#  time spent for once
a=time.time()
print fractal_dimension_gray_np_parallel_scale(image_sample)
b=time.time()
timer(a,b)

2.4055603640540757
00:00:11.84


Parallell Split Image

In [0]:
#
# Split image in blocks with size (M,N)
#
#

def split_images_block(image,block=(256,256)):
    ### Initial quantity
    M,N = image.shape #
    m,n = block #
    Mim, Nin = M/m, N/n # number of integer blocks available
    Mrm, Nrn = M%m, N%n # number of over pixels
    #print Mrm, Nrn
    im=np.zeros((M+m,N+n))
    im[:M,:N]=image[:,:]
    
    if Mrm==0:
        dm=0
    else:
        dm=1
    if Nrn==0:
        dn=0
    else:
        dn=1
        
    range_m=np.arange(Mim+dm)
    range_n=np.arange(Nin+dn)
    
    sample = () # inicialize output
    
    ########
    cont=0
    for i in range_m:
        for j in range_n:
            im_slice=im[m*i:m*i+m,n*j:n*j+n]
            sample+=((im_slice),)
    return np.array(sample).astype(np.int)


In [0]:
def Sizes_FDG(M=256,block=(256,256),step=2):
    #
    #
    dim_h,dim_v=block
    # Minimal dimension of image
    p = min((dim_h,dim_v,M))
    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))
    # Extract the exponent
    n = int(np.log(n)/np.log(2))
    # Build successive box sizes (from 2**n down to 2**1)
    return 2**np.arange(n, 0, -1*step)
    

In [0]:
def Counts_py_FDG(image,sizes,M=256,block=(256,256),step=2):
    #
    dim_h,dim_v=block
    #
    ind=image_3d_no_zero(image)
    #
    #
    Ns = []
    for size in sizes:
        H, edges=np.histogramdd(ind, bins=(np.arange(0,dim_h+size,size),np.arange(0,dim_v+size,size),np.arange(0,M+size,size)))
        Ns.append(np.sum(H>0))
    return Ns

In [0]:
def Loop_py_FDG(image,M=256,block=(256,256),step=2,cores=1):
    
    assert(len(image.shape) == 2)
    
    #Sizes
    sizes=Sizes_FDG(M,block,step)
    
    im_slice=split_images_block(image,block)
    
    Ns=Parallel(n_jobs=cores)(delayed(Counts_py_FDG)(node,sizes,M,block,step) for node in im_slice)
    counts=np.sum(Ns,axis=0)
    
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)

    return -coeffs[0]
    

In [0]:
#  time spent for once
a=time.time()
print Loop_py_FDG(image_sample,cores=2)
b=time.time()
timer(a,b)

2.4055603640540757
00:00:07.84


Edge Density - With Total Variation ( TV )
==========

Code inspired by page:  http://numba.pydata.org/numba-doc/0.15.1/examples.html

More about TV : https://link.springer.com/referenceworkentry/10.1007%2F978-0-387-92920-0_23

Scikit-Image Sobel vs Numba Exemple Vs Numba Parallel
-------------------------------------

Scikit-Image Sobel



In [0]:
from skimage.filters import sobel_h, sobel_v

In [0]:
def tv_image_sobel_sk(x):
    pv,ph=x.shape
    vdiff = sobel_v(x)
    hdiff = sobel_h(x)
    v_norma = np.sqrt(vdiff**2+hdiff**2)
    return (100*np.sum((v_norma-v_norma.min())/(v_norma.max()-v_norma.min())/(pv*ph)))

In [0]:
# In -1 to 1 interval
image_sample=rgb2gray(data.imread(fname=data_folder+"/EP-00-00012_0119_0001.bmp")).astype(np.float32)

In [0]:
# time spent for once
a=time.time()
print tv_image_sobel_sk(image_sample)
b=time.time()
timer(a,b)

5.993466075656425
00:00:01.14


In [0]:
#Nine times
a=time.time()
for i in range(1,9):
    image_sample=rgb2gray(data.imread(fname=data_folder+"/EP-00-00012_0119_000"+str(i)+".bmp")).astype(np.float32)
    x=tv_image_sobel_sk(image_sample)
    print x
b=time.time()
timer(a,b)

5.993466075656425
9.208419235621731
4.381172459683846
3.7731116177527086
4.124485259696868
5.787335948452389
5.369725139192727
4.305995229615803
00:00:18.17


Numba Exemple

In [0]:
import numba as nb

In [0]:
def filter2d(image, filt):
    M, N = image.shape
    Mf, Nf = filt.shape
    Mf2 = Mf // 2
    Nf2 = Nf // 2
    result = np.zeros_like(image)
    for i in range(Mf2, M - Mf2):
        for j in range(Nf2, N - Nf2):
            num = 0.0
            for ii in range(Mf):
                for jj in range(Nf):
                    num += (filt[Mf-1-ii, Nf-1-jj] * image[i-Mf2+ii, j-Nf2+jj])
            result[i, j] = num
    return result

# This kind of quadruply-nested for-loop is going to be quite slow.
# Using Numba we can compile this code to LLVM which then gets
# compiled to machine code:

fastfilter_2d = nb.jit(nb.float_[:,:](nb.float_[:,:], nb.float_[:,:]))(filter2d)

# Now fastfilter_2d runs at speeds as if you had first translated
# it to C, compiled the code and wrapped it with Python

In [0]:
# Sobel Masks
Gx=np.array(((-1,0,+1),(-2,0,+2),(-1,0,+1)),dtype=np.float32)
Gy=Gx.T[::-1]

In [0]:
def tv_image_sobel_numba(x):
    pv,ph=x.shape
    vdiff = fastfilter_2d(x,Gx)
    hdiff = fastfilter_2d(x,Gy)
    v_norma =  np.sqrt(vdiff**2+hdiff**2)
    return (100*np.sum((v_norma-v_norma.min())/(v_norma.max()-v_norma.min())/(pv*ph)))

In [0]:
# time spent for once
a=time.time()
print tv_image_sobel_numba(image_sample)
b=time.time()
timer(a,b)

4.305995357426297
00:00:01.65


In [0]:
#Nine times
a=time.time()
for i in range(1,9):
    image_sample=rgb2gray(data.imread(fname=data_folder+"/EP-00-00012_0119_000"+str(i)+".bmp")).astype(np.float32)
    x=tv_image_sobel_numba(image_sample)
    print x
b=time.time()
timer(a,b)

5.993466271142471
9.208419229476771
4.381172428060171
3.7731117041986715
4.1244853231298135
5.787335961742245
5.369725168580316
4.305995357426297
00:00:22.08


Use filter to make a TV with or without parallelism

In [0]:
@nb.jit()

def fast_tv_numba(image, filt):
    filt_x=filt[:,:]
    filt_y=filt.T[::-1]
    M, N = image.shape
    Mf, Nf = filt.shape
    Mf2 = Mf // 2
    Nf2 = Nf // 2
    result = np.zeros_like(image)
    for i in range(Mf2, M - Mf2):
        for j in range(Nf2, N - Nf2):
            num_x = 0.0
            num_y = 0.0
            for ii in range(Mf):
                for jj in range(Nf):
                    num_x += (filt_x[Mf-1-ii, Nf-1-jj] * image[i-Mf2+ii, j-Nf2+jj])
                    num_y += (filt_y[Mf-1-ii, Nf-1-jj] * image[i-Mf2+ii, j-Nf2+jj])
            result[i, j] = np.sqrt(num_x**2+num_y**2)
    return (100*np.sum((result-result.min())/(result.max()-result.min())/(M*N)))

In [0]:
#Nine times
a=time.time()
for i in range(1,9):
    image_sample=rgb2gray(data.imread(fname=data_folder+"/EP-00-00012_0119_000"+str(i)+".bmp"))
    x=fast_tv_numba(image_sample,Gx)
    print x
b=time.time()
timer(a,b)

5.99346597215
9.20841937218
4.38117251231
3.77311153713
4.12448516586
5.78733602231
5.36972511191
4.30599522467
00:00:17.38


In [0]:
@nb.njit(parallel=True)

def fast_tv_numba_par(image, filt):
    filt_x=filt[:,:]
    filt_y=filt.T[::-1]
    M, N = image.shape
    Mf, Nf = filt.shape
    Mf2 = Mf // 2
    Nf2 = Nf // 2
    result = np.zeros_like(image)
    for i in nb.prange(Mf2, M - Mf2):
        for j in range(Nf2, N - Nf2):
            num_x = 0.0
            num_y = 0.0
            for ii in range(Mf):
                for jj in range(Nf):
                    num_x += (filt_x[Mf-1-ii, Nf-1-jj] * image[i-Mf2+ii, j-Nf2+jj])
                    num_y += (filt_y[Mf-1-ii, Nf-1-jj] * image[i-Mf2+ii, j-Nf2+jj])
            result[i, j] = np.sqrt(num_x**2+num_y**2)
    return (100*np.sum((result-result.min())/(result.max()-result.min())/(M*N)))

In [0]:
#Nine times
a=time.time()
for i in range(1,9):
    image_sample=rgb2gray(data.imread(fname=data_folder+"/EP-00-00012_0119_000"+str(i)+".bmp"))
    x=fast_tv_numba_par(image_sample,Gx)
    print x
b=time.time()
timer(a,b)

5.99346597216
9.20841937214
4.38117251237
3.77311153709
4.1244851659
5.78733602229
5.36972511191
4.30599522466
00:00:18.23


Multiproposal NR Method - Robust Curvelet (RC)
======================

Code for Download by page:  https://github.com/rgiostri/robustcurvelet

More about RC :  https://arxiv.org/abs/1902.03842

